In [8]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from tqdm import tqdm


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.fc1 = nn.Linear(2048, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


class GradientStocker:
    def __init__(self, model_names):
        for item in model_names:
            setattr(self, item, 0)

    def get_attributes(self):
        return self.__dict__

    def add_gradient(self, model):
        for name, param in model.named_parameters():
            setattr(self, name, getattr(self, name) + param.grad.data.cpu())

    def euclidian_distance(self, grad_current_agent):
        """Computes the relative euclidean distance of the flattened tensor between the current model and the global model"""
        flattened_grad_selected = self.flatten(list(self.get_attributes().values()))
        flattened_grad_current = self.flatten(list(grad_current_agent.get_attributes().values()))
        return torch.dist(flattened_grad_selected, flattened_grad_current, 2) / torch.norm(flattened_grad_selected, 2)

    def flatten(self, gradient_list):
        """Returns an aggregated tensor of all the gradients for one model"""
        gradients = list(map(lambda g : torch.flatten(g), gradient_list))
        return torch.cat(gradients, 0)


def client_update(client_model, optimizer, train_loader, epoch=5):
    """Train a client_model on the train_loder data."""
    model_names = []
    for name, param in client_model.named_parameters():
        model_names.append(name)
    gradient_stocker = GradientStocker(model_names)
    
    for e in range(epoch):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = client_model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            gradient_stocker.add_gradient(client_model)
    return loss.item(), gradient_stocker


def weighted_average_gradients(gradients, weights):
    """Compute the weighted average gradient."""
    weighted_averages = {}
    for key in gradients[0].get_attributes().keys():
        weighted_averages[key] = weighted_average_from_key(key, gradients, weights)
    return weighted_averages

def weighted_average_from_key(key, gradients, weights):
    n = 0
    d = 0 
    for idx, g_dict in enumerate(gradients) :
        n += g_dict.get_attributes()[key] * weights[idx]
        d += weights[idx]
    return n / d

def compute_weight(alpha_prev, round, relative_distance, data_size, batch_size, distance_penalty, size_penalty):
    """Computes the weight alpha for round r"""
    size_factor = (1 + size_penalty * math.floor(((round - 1) * batch_size) / data_size)) 
    distance_factor = distance_penalty * relative_distance
    alpha = alpha_prev - size_factor * distance_factor 
    return max(0,alpha)

def update_grad(model, gradient, alpha): 
    for name, param in model.named_parameters():
        param.data -= gradient[name].cuda() * alpha
    return model 

def share_weight_erosion_model(shared_model, client_models):
    for model in client_models:
        model.load_state_dict(shared_model.state_dict())

def evaluate(global_model, data_loader):
    """Compute loss and accuracy of a model on a data_loader."""
    global_model.eval()
    loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.cuda(), target.cuda()
            output = global_model(data)
            loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    loss /= len(data_loader.dataset)
    acc = correct / len(data_loader.dataset)

    return loss, acc

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
distance_penalty = 0.05
size_penalty = 2

def runWeightErosion(train_loader,test_loader,num_clients,batch_size,selected_agent_index,num_rounds,epochs):

    print("=== Weight Erosion ===")
    np.set_printoptions(precision=3)

    # Instantiate models and optimizers
    shared_model = Net().cuda()
    client_models = [Net().cuda() for _ in range(num_clients)]
    for model in client_models:
        model.load_state_dict(shared_model.state_dict())

    opt = [optim.SGD(model.parameters(), lr=0.1) for model in client_models]

    grad_vector = [None for _ in range(num_clients)]
    weight_vector = np.ones(num_clients)

    for r in range(num_rounds):

        print('%d-th round' % r)

        # client update
        loss = np.zeros(num_clients)
        for i in range(num_clients):
            loss_tmp, grad_vector[i] = client_update(client_models[i], opt[i], train_loader[i], epoch=epochs)
            loss[i] = loss_tmp
            d_rel = grad_vector[selected_agent_index].euclidian_distance(grad_vector[i])
            weight_vector[i] = compute_weight(weight_vector[i], r + 1, d_rel, len(train_loader[i]), batch_size, distance_penalty, size_penalty)


        # Weight Erosion Scheme
        weighted_mean_gradient = weighted_average_gradients(grad_vector, weight_vector)
        shared_model = update_grad(shared_model, weighted_mean_gradient, 0.1)

        # Share model to all agents
        share_weight_erosion_model(shared_model, client_models)

        # Evalutate on the global test set (for now)
        test_loss, acc = evaluate(shared_model, test_loader)


        print(f"Weight : {weight_vector}")
        print(f"Loss   : {loss}")
        print('Test loss %0.3g | Test acc: %0.3f \n' % (test_loss, acc))

In [4]:
def runFederated(train_loader,test_loader,num_clients,batch_size,selected_agent_index,num_rounds,epochs):

    print("=== Federated ===")
    np.set_printoptions(precision=3)

    # Instantiate models and optimizers
    shared_model = Net().cuda()
    client_models = [Net().cuda() for _ in range(num_clients)]
    for model in client_models:
        model.load_state_dict(shared_model.state_dict())

    opt = [optim.SGD(model.parameters(), lr=0.1) for model in client_models]

    grad_vector = [None for _ in range(num_clients)]
    weight_vector = np.ones(num_clients)

    for r in range(num_rounds):

        print('%d-th round' % r)

        # client update
        loss = np.zeros(num_clients)
        for i in range(num_clients):
            loss_tmp, grad_vector[i] = client_update(client_models[i], opt[i], train_loader[i], epoch=epochs)
            loss[i] = loss_tmp
            weight_vector[i] = 1/num_clients


        # Weight Erosion Scheme
        weighted_mean_gradient = weighted_average_gradients(grad_vector, weight_vector)
        shared_model = update_grad(shared_model, weighted_mean_gradient, 0.1)

        # Share model to all agents
        share_weight_erosion_model(shared_model, client_models)

        # Evalutate on the global test set (for now)
        test_loss, acc = evaluate(shared_model, test_loader)

        print(f"Loss   : {loss}")
        print('Test loss %0.3g | Test acc: %0.3f\n' % (test_loss, acc))

In [5]:
def runLocal(train_loader,test_loader,num_clients,batch_size,selected_agent_index,epochs):

    print("=== Local ===")
    np.set_printoptions(precision=3)

    # Instantiate models and optimizers
    shared_model = Net().cuda()
    client_models = [Net().cuda() for _ in range(num_clients)]
    for model in client_models:
        model.load_state_dict(shared_model.state_dict())

    opt = [optim.SGD(model.parameters(), lr=0.1) for model in client_models]

    grad_vector = [None for _ in range(num_clients)]
    weight_vector = np.ones(num_clients)

    # client update
    loss = np.zeros(num_clients)
    for i in range(num_clients):
        print('%d-th Client' % i)
        loss_tmp, grad_vector[i] = client_update(client_models[i], opt[i], train_loader[i], epoch=epochs)
        loss[i] = loss_tmp

    # Evalutate on the global test set (for now)
    test_loss, acc = evaluate(client_models[selected_agent_index], test_loader)

    print(f"Loss   : {loss}")
    print('Test loss %0.3g | Test acc: %0.3f\n' % (test_loss, acc))

In [6]:
def getLoader(num_clients,batch_size,homogeneity):
    if homogeneity:
        traindata = datasets.MNIST('./data', train=True, download=True,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))]))
        traindata_split = torch.utils.data.random_split(traindata, [int(traindata.data.shape[0] / num_clients) for _ in range(num_clients)])
        train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]
        test_loader = torch.utils.data.DataLoader(datasets.MNIST('./data', train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),
            (0.3081,))])), batch_size=batch_size, shuffle=True)

        return train_loader, test_loader
    else:
        traindata = datasets.MNIST('./data', train=True, download=True,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))]))
        target_labels = torch.stack([traindata.targets == i for i in range(10)])
        target_labels_split = []
        for i in range(5):
            target_labels_split += torch.split(torch.where(target_labels[(2 * i):(2 * (i + 1))].sum(0))[0], int(60000 / num_clients))
        traindata_split = [torch.utils.data.Subset(traindata, tl) for tl in target_labels_split]
        train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]
        test_loader = torch.utils.data.DataLoader(datasets.MNIST('./data', train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])), batch_size=batch_size, shuffle=True)

        return train_loader, test_loader

In [11]:
from cumulator import base

#=== parameters for Schemes
selected_agent_index = 0
num_rounds = 10
epochs = 1

#=== parameters for training and testing
num_clients = 5
batch_size = 32
homogeneity = True

train_loader, test_loader = getLoader(num_clients,batch_size,homogeneity) #lot of change needed 

#cumulator not done yet
cumulator = base.Cumulator()
cumulator.on()

runWeightErosion(train_loader,test_loader,num_clients,batch_size,selected_agent_index,num_rounds,epochs)
runFederated(train_loader,test_loader,num_clients,batch_size,selected_agent_index,num_rounds,epochs)
runLocal(train_loader,test_loader,num_clients,batch_size,selected_agent_index,num_rounds*epochs)

cumulator.off()
dontknow = cumulator.computation_costs()

=== Weight Erosion ===
0-th round
Weight : [1.    0.974 0.97  0.974 0.974]
Loss   : [0.168 0.258 0.109 0.302 0.24 ]
Test loss 0.181 | Test acc: 0.944 

1-th round
Weight : [1.    0.928 0.925 0.928 0.927]
Loss   : [0.024 0.141 0.331 0.256 0.157]
Test loss 0.0872 | Test acc: 0.973 

2-th round
Weight : [1.    0.872 0.868 0.871 0.87 ]
Loss   : [0.016 0.132 0.046 0.303 0.017]
Test loss 0.0686 | Test acc: 0.978 

3-th round
Weight : [1.    0.81  0.805 0.809 0.807]
Loss   : [0.08  0.145 0.028 0.049 0.173]
Test loss 0.0583 | Test acc: 0.981 

4-th round
Weight : [1.    0.745 0.738 0.742 0.743]
Loss   : [0.039 0.054 0.065 0.092 0.006]
Test loss 0.052 | Test acc: 0.981 

5-th round
Weight : [1.    0.677 0.67  0.674 0.674]
Loss   : [0.195 0.05  0.052 0.142 0.057]
Test loss 0.0461 | Test acc: 0.985 

6-th round
Weight : [1.    0.608 0.6   0.605 0.606]
Loss   : [0.324 0.013 0.047 0.001 0.018]
Test loss 0.0414 | Test acc: 0.986 

7-th round
Weight : [1.    0.537 0.53  0.535 0.536]
Loss   : [0.001 0

In [3]:
# IID case: all the clients have images of all the classes

# Hyperparameters

num_clients = 5
num_rounds = 10
epochs = 1
batch_size = 32
distance_penalty = 0.05
size_penalty = 2
selected_agent_index = 0

# weight_vector

weight_vector = np.ones(num_clients)

# Creating decentralized datasets

traindata = datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
                       )
traindata_split = torch.utils.data.random_split(traindata, [int(traindata.data.shape[0] / num_clients) for _ in range(num_clients)])
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
        ), batch_size=batch_size, shuffle=True)


# Instantiate models and optimizers
shared_model = Net().cuda()
client_models = [Net().cuda() for _ in range(num_clients)]
for model in client_models:
    model.load_state_dict(shared_model.state_dict())

opt = [optim.SGD(model.parameters(), lr=0.1) for model in client_models]

grad_vector = [None for _ in range(num_clients)]
# Runnining Weight Erosion 

for r in range(num_rounds):
    # client update
    loss = 0
    for i in range(num_clients):
        loss_tmp, grad_vector[i] = client_update(client_models[i], opt[i], train_loader[i], epoch=epochs)
        loss += loss_tmp
        d_rel = grad_vector[0].euclidian_distance(grad_vector[i])
        weight_vector[i] = compute_weight(weight_vector[i], r + 1, d_rel, len(train_loader[i]), batch_size, distance_penalty, size_penalty)
        print(f"Weight alpha for agent {i} : {weight_vector[i]}")
    
    # Weight Erosion Scheme 
    weighted_mean_gradient = weighted_average_gradients(grad_vector, weight_vector)
    shared_model = update_grad(shared_model, weighted_mean_gradient, 0.1)
    
    # Share model to all agents 
    share_weight_erosion_model(shared_model, client_models)
    
    # Evalutate on the global test set (for now)
    test_loss, acc = evaluate(shared_model, test_loader)

    print('%d-th round' % r)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f' % (loss / num_clients, test_loss, acc))

Weight alpha for agent 0 : 1.0
Weight alpha for agent 1 : 0.9728857278823853
Weight alpha for agent 2 : 0.9722303152084351
Weight alpha for agent 3 : 0.9726162552833557
Weight alpha for agent 4 : 0.9697747826576233
0-th round
average train loss 0.18 | test loss 0.143 | test acc: 0.958
Weight alpha for agent 0 : 1.0
Weight alpha for agent 1 : 0.9249209761619568
Weight alpha for agent 2 : 0.9247506260871887
Weight alpha for agent 3 : 0.9261295199394226
Weight alpha for agent 4 : 0.9224545359611511
1-th round
average train loss 0.181 | test loss 0.0819 | test acc: 0.973
Weight alpha for agent 0 : 1.0
Weight alpha for agent 1 : 0.8674433827400208
Weight alpha for agent 2 : 0.8652442693710327
Weight alpha for agent 3 : 0.8685817718505859
Weight alpha for agent 4 : 0.8642969727516174
2-th round
average train loss 0.148 | test loss 0.0639 | test acc: 0.979
Weight alpha for agent 0 : 1.0
Weight alpha for agent 1 : 0.8044438362121582
Weight alpha for agent 2 : 0.8012921810150146
Weight alpha fo

In [4]:
# NON-IID case: every client has images of two categories chosen from [0, 1], [2, 3], [4, 5], [6, 7], or [8, 9].

# Hyperparameters

num_clients = 5
num_rounds = 10
epochs = 1
batch_size = 32
distance_penalty = 0.05
size_penalty = 2
selected_agent_index = 0

# weight_vector

weight_vector = np.ones(num_clients)

# Creating decentralized datasets

traindata = datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
                       )
target_labels = torch.stack([traindata.targets == i for i in range(10)])
target_labels_split = []
for i in range(5):
    target_labels_split += torch.split(torch.where(target_labels[(2 * i):(2 * (i + 1))].sum(0))[0], int(60000 / num_clients))
traindata_split = [torch.utils.data.Subset(traindata, tl) for tl in target_labels_split]
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
        ), batch_size=batch_size, shuffle=True)

# Instantiate models and optimizers

to_share_model = Net().cuda()
client_models = [Net().cuda() for _ in range(num_clients)]
for model in client_models:
    model.load_state_dict(to_share_model.state_dict())

opt = [optim.SGD(model.parameters(), lr=0.1) for model in client_models]


for r in range(num_rounds):
    # client update
    loss = 0
    for i in range(num_clients):
        loss_tmp, grad_vector[i] = client_update(client_models[i], opt[i], train_loader[i], epoch=epochs)
        loss += loss_tmp
        d_rel = relative_distance_vector(grad_vector[selected_agent_index], grad_vector[i])
        weight_vector[i] = compute_weight(weight_vector[i], r + 1, d_rel, len(train_loader[i]), batch_size, distance_penalty, size_penalty)
        print(f"Weight alpha for agent {i} : {weight_vector[i]}")
    
    # Weight Erosion Scheme 
    weighted_mean_gradient = weighted_average_gradients(grad_vector, weight_vector)
    to_share_model = update_grad(to_share_model, weighted_mean_gradient, 0.1)
    
    # Share model to all agents 
    share_weight_erosion_model(to_share_model, client_models)
    
    # Evalutate on the global test set (for now)
    test_loss, acc = evaluate(to_share_model, test_loader)

    print('%d-th round' % r)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f' % (loss / num_clients, test_loss, acc))

NameError: name 'relative_distance_vector' is not defined

In [ ]:
# IID case: all the clients have images of all the classes

# Hyperparameters

num_clients = 5
num_rounds = 5
epochs = 1
batch_size = 32

# Communication matrix

comm_matrix = np.ones((num_clients, num_clients)) / num_clients
# comm_matrix = np.eye(num_clients)

# Creating decentralized datasets

traindata = datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
                       )
traindata_split = torch.utils.data.random_split(traindata, [int(traindata.data.shape[0] / num_clients) for _ in range(num_clients)])
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
        ), batch_size=batch_size, shuffle=True)

# Instantiate models and optimizers

global_model = Net().cuda()
client_models = [Net().cuda() for _ in range(num_clients)]
for model in client_models:
    model.load_state_dict(global_model.state_dict())

opt = [optim.SGD(model.parameters(), lr=0.1) for model in client_models]

# Runnining Decentralized training

for r in range(num_rounds):
    # client update
    loss = 0
    for i in range(num_clients):
        loss += client_update(client_models[i], opt[i], train_loader[i], epoch=epochs)
    
    # diffuse params
    diffuse_params(client_models, comm_matrix)

    average_models(global_model, client_models)
    test_loss, acc = evaluate(global_model, test_loader)
    
    print('%d-th round' % r)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f' % (loss / num_clients, test_loss, acc))